In [1]:
import hipscat
import healpy as hp
import pandas as pd
from tqdm import tqdm
from hipscat.inspection import plot_pixel_list
from hipscat.pixel_math import HealpixPixel
import numpy as np
import matplotlib.pyplot as plt

In [2]:
step = 2

In [3]:
import gundam.cflibfor as cff
import numpy as np
import pandas as pd
from gundam import gundam
from hipscat.catalog.catalog_info import CatalogInfo
from munch import Munch
from corrgi.utils import project_coordinates


In [4]:
params = gundam.packpars(kind="acf")
params.dsept = 0.1
params.nsept = 33
params.septmin = 0.01

bins, _ = gundam.makebins(
            params.nsept, params.septmin, params.dsept, params.logsept
        )

Missing outfn: default to run001


In [36]:
gaia_full_partition_frame = pd.read_csv("gaia_partition_info.csv")
gaia_full_partition_list = [
            HealpixPixel(order, pixel)
            for order, pixel in zip(
                gaia_full_partition_frame["Norder"],
                gaia_full_partition_frame["Npix"],
            )
        ]
num_partitions = len(gaia_full_partition_list)
# num_partitions = 3
print(num_partitions, "partitions")
print(int(.5 * num_partitions * (num_partitions - 1)), "iterations")

3933 partitions
7732278 iterations


In [31]:
## i know this is dumb. i don't care.

def calc_num_bins(bounds_a, bounds_b):
    a_x, a_y, a_z = project_coordinates(ra=bounds_a[0], dec=bounds_a[1])
    b_x, b_y, b_z = project_coordinates(ra=bounds_b[0], dec=bounds_b[1])
    args = [
            len(a_x),  # number of particles of the left partition
            a_x, a_y, a_z,  # X,Y,Z coordinates of particles
            len(b_x),  # number of particles of the right partition
            b_x, b_y, b_z,  # X,Y,Z coordinates of particles
            params.nsept,  # number of angular separation bins
            bins,  # bins in angular separation [deg]
        ]
    bins_populated = cff.mod.th_C_naiveway(*args)
    populated_bins = np.flatnonzero(bins_populated)
    overlap_width = 0
    if len(populated_bins) > 0:
        overlap_width = populated_bins.max() - populated_bins.min() + 1
    return overlap_width

In [37]:
num_bins = []

for a in tqdm(range(0, num_partitions)):
    for b in range(a, num_partitions):
        bounds_a = hp.vec2dir(hp.boundaries(2**gaia_full_partition_list[a].order, gaia_full_partition_list[a].pixel, step=step, nest=True), lonlat=True)
        bounds_b = hp.vec2dir(hp.boundaries(2**gaia_full_partition_list[b].order, gaia_full_partition_list[b].pixel, step=step, nest=True), lonlat=True)
        num_bins.append(calc_num_bins(bounds_a, bounds_b))

100%|██████████| 3933/3933 [05:16<00:00, 12.43it/s] 


In [38]:
len(num_bins)

7736211

In [40]:
pix_a_order = []
pix_a_pixel = []
pix_b_order = []
pix_b_pixel = []

for a in tqdm(range(0, num_partitions)):
    for b in range(a, num_partitions):
        pix_a_order.append(gaia_full_partition_list[a].order)
        pix_a_pixel.append(gaia_full_partition_list[a].pixel)
        pix_b_order.append(gaia_full_partition_list[b].order)
        pix_b_pixel.append(gaia_full_partition_list[b].pixel)


big_beautiful_frame = pd.DataFrame({"Norder_a": pix_a_order,
                                    "Npix_a": pix_a_pixel,
                                    "Norder_b": pix_b_order,
                                    "Npix_b": pix_b_pixel,
                                    "num_bins": num_bins,
                                    })
big_beautiful_frame.to_csv("num_bins.csv", index=False)

100%|██████████| 3933/3933 [00:01<00:00, 2251.26it/s]
